# Import library and dataset from sklearn

In [1]:
import torch, os
from torch import nn
import numpy as np
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.datasets import load_breast_cancer
dataset = load_breast_cancer()
X = dataset.data
y = dataset.target

# Change the dataset to tensor and split train and test dataset 

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_a = torch.from_numpy(X.astype(np.float32)).to(device)
y_a = (torch.from_numpy(y.astype(np.float32))).view(-1,1).to(device)

dataset = TensorDataset(X_a, y_a)

test_ratio = 0.2  # 20% data split into test dataset
n_sample,n_feature = X_a.shape
test_size = int(n_sample * test_ratio)
train_size = n_sample - test_size
train_ds, test_ds = random_split(dataset,[train_size, test_size])
train_dataloader = DataLoader(train_ds, batch_size=train_size, shuffle=True)
test_dataloader = DataLoader(test_ds, batch_size=test_size)

# Create a NN model with 2 hidden layer
first layer = 10

second layer = 5

In [3]:
class LogisticNN(nn.Module):
    def __init__(self):
        super(LogisticNN, self).__init__()
        layer1_out = 10
        layer2_out = 5
        self.layer1 = nn.Linear(30,layer1_out,bias=True)
        self.layer2 = nn.Linear(layer1_out,layer2_out,bias=True)
        self.output = nn.Linear(layer2_out,1,bias=True)

    def forward(self, x):
        z = torch.relu(self.layer1(x))
        z = torch.relu(self.layer2(z))
        z = torch.sigmoid(self.output(z))
        return z

# Load, Train, and Save the model
loss model = Binary Cross Entropy

Optimizer = Adam optimizer

In [4]:
model = LogisticNN().to(device)
if os.path.exists('breast_cancer.pt'):
    model.load_state_dict(torch.load('breast_cancer.pt',map_location=device))
    epoch_max = 100
    report = 10
else:
    epoch_max = 10000
    report = 500
loss = nn.BCELoss()
rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(),lr=rate,weight_decay=1e-3)
for i in range(epoch_max):
    for x_batch_train, y_batch_train in train_dataloader:
        z = model(x_batch_train)
        J = loss(z,y_batch_train)
        J.backward()
        optimizer.step()
        optimizer.zero_grad()
    if i % report  == 0:
        with torch.no_grad():
            acc = (1-(z.round()-y_batch_train).abs().mean().item())*100
            for x_batch_test, y_batch_test in test_dataloader:
                zx = model(x_batch_test)
                acc_test = (1-(zx-y_batch_test).abs().mean().item())*100
        print(f'Epoch {i}: Cost = {J.item():.4f}, train acc = {acc:.4f}%, test acc = {acc_test:.4f}%')

torch.save(model.state_dict(),'breast_cancer.pt')

Epoch 0: Cost = 0.1138, train acc = 98.0263%, test acc = 90.6274%
Epoch 10: Cost = 0.1662, train acc = 96.9298%, test acc = 90.1013%
Epoch 20: Cost = 0.1267, train acc = 97.5877%, test acc = 94.4433%
Epoch 30: Cost = 0.1115, train acc = 98.0263%, test acc = 95.7582%
Epoch 40: Cost = 0.1062, train acc = 98.0263%, test acc = 96.2405%
Epoch 50: Cost = 0.1004, train acc = 97.8070%, test acc = 96.6678%
Epoch 60: Cost = 0.0982, train acc = 97.5877%, test acc = 96.7935%
Epoch 70: Cost = 0.0978, train acc = 97.8070%, test acc = 96.7777%
Epoch 80: Cost = 0.0971, train acc = 97.8070%, test acc = 96.7700%
Epoch 90: Cost = 0.0965, train acc = 97.5877%, test acc = 96.8612%


# Report of the neural network training
createing confusion matrix, accuracy, recall, precision, and F1 score. 

In [28]:
with torch.no_grad():
    z = model(X_a).cpu().detach().numpy().flatten()
confusion_matrix = np.zeros((2,2),dtype=int)
z_val = z.round().astype(int)
for i,val in enumerate(z):
    confusion_matrix[y[i],z_val[i]] +=1

In [27]:
precision = confusion_matrix[0,0]/confusion_matrix[0,:].sum()
recall = confusion_matrix[0,0]/confusion_matrix[:,0].sum()
F1 = 2*precision*recall/(precision+recall)
accuracy = confusion_matrix.diagonal().sum() /confusion_matrix.sum()

print(f'The result show for all value that\
    \n\n Accuracy = {accuracy*100:.5f}%\
    \n Precision = {precision*100:.5f}%\
    \n Recall = {recall*100:.5f}%\
    \n F1 score = {F1:.4f}    ')

The result show for all value that    

 Accuracy = 97.53954%    
 Precision = 95.75472%    
 Recall = 97.59615%    
 F1 score = 0.9667    
